# * Sales Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [2]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-04-10, 10:57:36

   -> Summary DataFrame : 1126 rows, 18 columns

TDMDBPR : Disconnected


In [3]:
def my_metric_group(v_name):

    # Get : Parameter
    name = v_name
    result = ''
    
    if re.search('Gross Adds', name): result = 'Gross Adds'
    elif re.search('Inflow M1', name): result = 'Inflow M1'
    elif re.search('Inflow M2', name): result = 'Inflow M2'
    elif re.search('No of AP 1D', name): result = 'No of AP 1D'
    elif re.search('Amt of AP 1D', name): result = 'Amt of AP 1D'
    elif re.search('No of AP In Month', name): result = 'No of AP In Month'
    elif re.search('Amt of AP In Month', name): result = 'Amt of AP In Month'
    # elif re.search('CConnected : onsumer', name): result = 'Gross Adds - Con - Consumer'
    # elif re.search('Connected : DataService', name): result = 'Gross Adds - Con - DataService'
    # elif re.search('Registration : Consumer', name): result = 'Gross Adds - Reg - Consumer'
    # elif re.search('Registration : DataService', name): result = 'Gross Adds - Reg - DataService'
    else: result = 'Unknown' 
    
    return result

In [4]:
''' Add Columns '''

chk_src_df['CHANNEL_GRP'] = np.where(chk_src_df['METRIC_CD'].str.contains('CS$'), 'ALL', 'GEO')
chk_src_df['LOCATION_GRP'] = np.where(chk_src_df['METRIC_NAME'].str.contains('Location'), 'Customer', 'Sales')
chk_src_df['METRIC_GRP'] = chk_src_df.apply(lambda x: my_metric_group(str(x['METRIC_NAME'])), axis=1)
# tmp_df['MAIN_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')) | (tmp_df['METRIC_NAME'].str.contains('New Revenue|Existing Revenue')), 'N', 'Y')
# chk_src_df.tail()

# chk_src_df.loc[chk_src_df['PRODUCT']=='TOL'].tail()

### KPIs Summary

In [5]:
''' PREPAID Summary : 14 rows, 24 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'PREPAID'
# channel_grp = 'GEO'

prepaid_df = chk_src_df.copy()
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==curr_mth]
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT']==product]
# prepaid_df = prepaid_df.loc[prepaid_df['CHANNEL_GRP']==channel_grp]

prepaid_df['ALL'] = prepaid_df['P']
prepaid_df['TRUE'] = np.where(prepaid_df['COMP_CD']=='TRUE', prepaid_df['P'], 0)
prepaid_df['DTAC'] = np.where(prepaid_df['COMP_CD']=='DTAC', prepaid_df['P'], 0)
prepaid_df['TMP_NAME'] = prepaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

prepaid_df = prepaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
prepaid_df = prepaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = prepaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    prepaid_df[col] = prepaid_df[col].apply(lambda x: format(x, ',.0f'))

prepaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D,2025-04-10 08:00:56,20250401,20250409,"91,367,726","50,751,282","40,616,444"
1,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D - GEO Channel,2025-04-10 08:00:56,20250401,20250409,"89,238,653","50,041,124","39,197,529"
2,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month,2025-04-10 08:00:56,20250401,20250409,"93,900,625","52,045,357","41,855,268"
3,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month - GEO Channel,2025-04-10 08:00:56,20250401,20250409,"91,739,779","51,331,937","40,407,842"
4,PREPAID,Gross Adds,Prepaid Gross Adds,2025-04-10 08:00:56,20250401,20250409,"465,092","249,680","215,412"
5,PREPAID,Gross Adds,Prepaid Gross Adds - GEO Channel,2025-04-10 08:00:56,20250401,20250409,"458,584","247,878","210,706"
6,PREPAID,Inflow M1,Prepaid Inflow M1,2025-04-10 08:00:59,20250401,20250407,"85,719,771","43,130,600","42,589,171"
7,PREPAID,Inflow M1,Prepaid Inflow M1 - GEO Channel,2025-04-10 08:00:59,20250401,20250407,"83,800,593","42,553,433","41,247,160"
8,PREPAID,Inflow M2,Prepaid Inflow M2,2025-04-10 08:00:56,20250401,20250407,"34,117,914","16,093,759","18,024,155"
9,PREPAID,Inflow M2,Prepaid Inflow M2 - GEO Channel,2025-04-10 08:00:56,20250401,20250407,"33,890,767","16,069,032","17,821,735"


In [6]:
''' POSTPAID Summary : 8 rows, 16 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'POSTPAID'
# channel_grp = 'GEO'

postpaid_df = chk_src_df.copy()
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==curr_mth]
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT']==product]
# postpaid_df = postpaid_df.loc[postpaid_df['CHANNEL_GRP']==channel_grp]

postpaid_df['ALL'] = postpaid_df['P']
postpaid_df['TRUE'] = np.where(postpaid_df['COMP_CD']=='TRUE', postpaid_df['P'], 0)
postpaid_df['DTAC'] = np.where(postpaid_df['COMP_CD']=='DTAC', postpaid_df['P'], 0)
postpaid_df['TMP_NAME'] = postpaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

postpaid_df = postpaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
postpaid_df = postpaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = postpaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    postpaid_df[col] = postpaid_df[col].apply(lambda x: format(x, ',.0f'))

postpaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,POSTPAID,Gross Adds,Postpaid Gross Adds B2B,2025-04-10 08:06:01,20250401,20250409,"8,077","5,005","3,072"
1,POSTPAID,Gross Adds,Postpaid Gross Adds B2B - GEO Channel,2025-04-10 08:06:01,20250401,20250409,"2,404","2,189",215
2,POSTPAID,Gross Adds,Postpaid Gross Adds B2C,2025-04-10 08:06:01,20250401,20250409,"63,986","50,672","13,314"
3,POSTPAID,Gross Adds,Postpaid Gross Adds B2C - GEO Channel,2025-04-10 08:06:01,20250401,20250409,"30,055","26,177","3,878"
4,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B,2025-04-10 08:06:01,20250401,20250409,"7,178,245","1,754,795","5,423,450"
5,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B - GEO Channel,2025-04-10 08:06:01,20250401,20250409,"996,850","890,115","106,735"
6,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C,2025-04-10 08:06:01,20250401,20250409,"28,569,831","22,967,441","5,602,390"
7,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C - GEO Channel,2025-04-10 08:06:01,20250401,20250409,"14,677,292","12,504,752","2,172,541"


In [7]:
''' TOL Summary : 20 KPIs'''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TOL'
# channel_grp = 'GEO'

tol_df = chk_src_df.copy()
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==curr_mth]
tol_df = tol_df.loc[tol_df['PRODUCT']==product]
# tol_df = tol_df.loc[tol_df['CHANNEL_GRP']==channel_grp]

tol_df = tol_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tol_df = tol_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tol_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tol_df[col] = tol_df[col].apply(lambda x: format(x, ',.0f'))

tol_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TOL,Gross Adds,TOL Gross Adds Connected : Consumer,2025-04-10 08:00:22,20250401,20250409,"16,292"
1,TOL,Gross Adds,TOL Gross Adds Connected : Consumer (Install L...,2025-04-10 08:00:22,20250401,20250409,"16,292"
2,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Channel,2025-04-10 08:00:22,20250401,20250409,"12,734"
3,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-04-10 08:00:22,20250401,20250409,"12,734"
4,TOL,Gross Adds,TOL Gross Adds Connected : DataService,2025-04-10 08:00:22,20250401,20250409,457
5,TOL,Gross Adds,TOL Gross Adds Connected : DataService (Instal...,2025-04-10 08:00:22,20250401,20250409,457
6,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-04-10 08:00:22,20250401,20250409,21
7,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-04-10 08:00:22,20250401,20250409,21
8,TOL,Gross Adds,TOL Gross Adds Registration : Consumer,2025-04-10 08:00:22,20250401,20250409,"23,529"
9,TOL,Gross Adds,TOL Gross Adds Registration : Consumer - GEO C...,2025-04-10 08:00:22,20250401,20250409,"17,609"


In [8]:
''' TVS Summary : 8 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TVS'
# channel_grp = 'GEO'

tvs_df = chk_src_df.copy()
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==curr_mth]
tvs_df = tvs_df.loc[tvs_df['PRODUCT']==product]
# tvs_df = tvs_df.loc[tvs_df['CHANNEL_GRP']==channel_grp]

tvs_df = tvs_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tvs_df = tvs_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tvs_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tvs_df[col] = tvs_df[col].apply(lambda x: format(x, ',.0f'))

tvs_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TVS,Gross Adds,TVS CMDU Gross Adds,2025-04-10 08:11:55,20250401,20250404,17
1,TVS,Gross Adds,TVS CMDU Gross Adds - GEO Channel,2025-04-10 08:11:55,20250401,20250404,5
2,TVS,Gross Adds,TVS Now Gross Adds,2025-04-10 08:11:55,20250401,20250408,402
3,TVS,Gross Adds,TVS Now Gross Adds - GEO Channel,2025-04-10 08:11:55,20250401,20250408,402
4,TVS,Inflow M1,TVS CMDU Inflow M1,2025-04-10 08:11:55,20250401,20250404,"167,357"
5,TVS,Inflow M1,TVS CMDU Inflow M1 - GEO Channel,2025-04-10 08:11:55,20250401,20250404,"36,517"
6,TVS,Inflow M1,TVS Now Inflow M1,2025-04-10 08:11:55,20250401,20250408,"110,539"
7,TVS,Inflow M1,TVS Now Inflow M1 - GEO Channel,2025-04-10 08:11:55,20250401,20250408,"110,539"


### Monitoring

In [9]:
''' Current Month Gross Adds '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH']==curr_mth]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-04-10 08:06:01,20250401,20250408,0,"13,314","13,199","13,199","13,199","13,199","13,199",7,4012,11945
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-04-10 08:06:01,20250401,20250408,0,"3,072",838,838,838,838,838,7,527,1141
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-04-10 08:06:01,20250401,20250409,0,"50,672","36,160","36,160","36,160","36,160","36,160",7,1449,8731
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-04-10 08:06:01,20250401,20250409,0,"5,005","2,191","2,191","2,191","2,191","2,191",7,507,1408
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-04-10 08:00:56,20250401,20250407,0,"215,412","184,391","184,391","204,292","207,296","205,991",7,5642,24695
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-04-10 08:00:56,20250401,20250409,0,"249,680","195,544","195,544","190,410","195,544","195,530",7,5438,31216
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-04-10 08:00:22,20250401,20250409,0,"16,292","12,734","12,734","12,607","12,734","12,734",7,1075,6836
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-04-10 08:00:22,20250401,20250409,0,457,21,21,21,21,21,7,61,83
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-04-10 08:00:22,20250401,20250409,0,"23,529","17,608","17,608","17,417","17,608","17,608",7,1096,7124
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-04-10 08:00:22,20250401,20250409,0,567,17,17,17,17,17,7,51,67


In [10]:
''' Average Gross Adds per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-04-10 08:06:01,20240101,20250331,0,"61,560","59,720","59,720","59,720","59,720","59,720",7,6204,697818
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-04-10 08:06:01,20240101,20250331,0,"10,051","5,364","5,364","5,364","5,364","5,364",7,1528,80454
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-04-10 08:06:01,20240101,20250331,0,"152,588","103,167","103,167","103,167","103,167","103,167",7,2211,438943
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-04-10 08:06:01,20240101,20250331,0,"25,021","8,124","8,124","8,124","8,124","8,124",7,781,64761
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-04-10 08:00:56,20240101,20250331,0,"1,347,209","1,287,777","1,287,777","1,283,901","1,301,854","1,293,953",7,7441,1684708
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-04-10 08:00:56,20240101,20250331,0,"1,138,166","940,606","940,606","918,524","940,606","940,541",7,5831,1789493
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-04-10 08:00:22,20240102,20250331,0,"46,948","33,562","33,562","34,737","35,069","35,055",7,1214,331274
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-04-10 08:00:22,20240102,20250331,0,"1,584",34,34,35,35,35,7,99,2156
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-04-10 08:00:22,20240101,20250331,0,"64,322","45,578","45,578","47,202","47,631","47,608",7,1243,348927
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-04-10 08:00:22,20240101,20250331,0,"3,354","1,033","1,033","1,025","1,043","1,043",7,620,25738


In [11]:
''' Current Month Inflow '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH']==curr_mth]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-04-10 08:06:01,20250401,20250408,0,"5,602,390","5,545,403","5,545,403","5,545,403","5,545,403","5,545,403",7,4012,11945
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-04-10 08:06:01,20250401,20250408,0,"5,423,450","394,284","394,284","394,284","394,284","394,284",7,527,1141
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-04-10 08:06:01,20250401,20250409,0,"22,967,441","18,015,657","18,015,657","18,015,657","18,015,657","18,015,657",7,1449,8731
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-04-10 08:06:01,20250401,20250409,0,"1,754,795","891,313","891,313","891,313","891,313","891,313",7,507,1408
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-04-10 08:00:59,20250401,20250407,0,"42,589,171","37,499,644","37,499,644","40,964,687","41,623,101","41,354,135",7,5626,28490
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-04-10 08:00:56,20250401,20250407,0,"18,024,155","15,330,586","15,330,586","17,581,920","17,805,177","17,653,717",7,6519,42623
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-04-10 08:00:59,20250401,20250407,0,"43,130,600","33,443,990","33,443,990","32,449,689","33,443,990","33,441,773",7,5295,28577
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-04-10 08:00:56,20250401,20250407,0,"16,093,759","15,753,126","15,753,126","15,442,036","15,753,126","15,751,806",7,5545,35501
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-04-10 08:00:22,20250401,20250409,0,"8,979,138","7,159,764","7,159,764","7,085,527","7,159,764","7,159,764",7,1075,6836
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-04-10 08:00:22,20250401,20250409,0,"545,247","58,800","58,800","58,800","58,800","58,800",7,61,83


In [12]:
''' Average Inflow per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-04-10 08:06:01,20240101,20250331,0,"399,215,852","390,237,912","390,237,912","390,237,912","390,237,912","390,237,912",7,6204,697818
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-04-10 08:06:01,20240101,20250331,0,"146,103,303","27,648,863","27,648,863","27,648,863","27,648,863","27,648,863",7,1528,80454
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-04-10 08:06:01,20240101,20250331,0,"963,707,573","742,499,974","742,499,974","742,499,974","742,499,974","742,499,974",7,2211,438943
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-04-10 08:06:01,20240101,20250331,0,"84,587,795","36,558,027","36,558,027","36,558,027","36,558,027","36,558,027",7,781,64761
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-04-10 08:00:59,20240101,20250331,0,"3,802,290,888","3,695,645,471","3,695,645,471","3,681,783,820","3,740,469,911","3,715,646,009",7,7418,2536145
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-04-10 08:00:56,20240201,20250331,0,"1,083,317,243","1,050,794,381","1,050,794,381","1,056,976,090","1,069,493,307","1,057,310,986",7,7294,2747604
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-04-10 08:00:59,20240101,20250331,0,"2,938,521,504","2,535,601,009","2,535,601,009","2,461,916,507","2,535,601,009","2,535,437,768",7,5800,2096987
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-04-10 08:00:56,20240101,20250331,0,"1,060,137,134","1,049,549,983","1,049,549,983","1,030,456,436","1,049,549,983","1,049,435,343",7,5739,2269687
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-04-10 08:00:22,20240102,20250331,0,"388,811,233","283,268,444","283,268,444","293,322,901","296,385,102","296,262,477",7,1214,331274
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-04-10 08:00:22,20240102,20250331,0,"29,369,420","1,591,398","1,591,398","1,627,784","1,630,883","1,630,883",7,99,2156


### All Metric

In [13]:
''' METRIC Summary '''

# src_t_sales_mth_df = chk_src_df.copy()
# src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')

' METRIC Summary '

In [14]:
''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
# src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')

' METRIC Current Month '

## Actual : Transaction
->  DIM_CORP_KPI

In [15]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-04-10, 11:00:15

   -> Daily DataFrame : 6700 rows, 12 columns

TDMDBPR : Disconnected


In [16]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 6700 rows, 12 columns


In [17]:
''' xB1S000101CS : Prepaid Gross Adds : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB1S000101CS', 'TB1S000101CS']
print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB1S000101CS_day_df = chk_src_df.copy()
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB1S000101CS_day_df = xB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB1S000101CS_day_df[col] = xB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB1S000101CS_day_df


DB1S000101CS : Prepaid Gross Adds : DTAC
TB1S000101CS : Prepaid Gross Adds : TMH

   -> v_metric_cd: ['DB1S000101CS', 'TB1S000101CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250301,2025-04-10 08:00:56,0,"39,049","33,586","33,586","36,743","37,299","37,084"
1,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250302,2025-04-10 08:00:56,0,"41,058","33,973","33,973","37,323","37,926","37,511"
2,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250303,2025-04-10 08:00:56,0,"35,155","30,340","30,340","33,580","34,452","34,293"
3,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250304,2025-04-10 08:00:56,0,"33,235","28,892","28,892","32,054","32,845","32,708"
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250305,2025-04-10 08:00:56,0,"34,208","29,933","29,933","33,083","33,779","33,646"
...,...,...,...,...,...,...,...,...,...,...,...,...
73,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250405,2025-04-10 08:00:56,0,"31,312","24,859","24,859","24,282","24,859","24,857"
74,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250406,2025-04-10 08:00:56,0,"27,757","21,959","21,959","21,293","21,959","21,957"
75,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250407,2025-04-10 08:00:56,0,"27,788","21,862","21,862","21,305","21,862","21,860"
76,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250408,2025-04-10 08:00:56,0,"29,639","23,121","23,121","22,534","23,121","23,121"


In [18]:
''' xB2S010100CS : Postpaid Gross Adds B2C : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB2S010100CS', 'TB2S010100CS']
print(f'\nDB2S010100CS : Postpaid Gross Adds : DTAC\nTB2S010100CS : Postpaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB2S010100CS_day_df = chk_src_df.copy()
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB2S010100CS_day_df = xB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB2S010100CS_day_df[col] = xB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB2S010100CS_day_df


DB2S010100CS : Postpaid Gross Adds : DTAC
TB2S010100CS : Postpaid Gross Adds : TMH

   -> v_metric_cd: ['DB2S010100CS', 'TB2S010100CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250301,2025-04-10 08:06:01,0,"1,143","1,138","1,138","1,138","1,138","1,138"
1,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250302,2025-04-10 08:06:01,0,888,887,887,887,887,887
2,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250303,2025-04-10 08:06:01,0,"1,362","1,354","1,354","1,354","1,354","1,354"
3,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250304,2025-04-10 08:06:01,0,"1,696","1,684","1,684","1,684","1,684","1,684"
4,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250305,2025-04-10 08:06:01,0,"1,383","1,373","1,373","1,373","1,373","1,373"
...,...,...,...,...,...,...,...,...,...,...,...,...
74,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250405,2025-04-10 08:06:01,0,"5,445","3,871","3,871","3,871","3,871","3,871"
75,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250406,2025-04-10 08:06:01,0,"5,930","4,424","4,424","4,424","4,424","4,424"
76,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250407,2025-04-10 08:06:01,0,"5,016","3,260","3,260","3,260","3,260","3,260"
77,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250408,2025-04-10 08:06:01,0,"4,964","3,327","3,327","3,327","3,327","3,327"


In [19]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_day = 20250301
v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250301,2025-04-10 08:00:22,0,"1,759","1,376","1,376","1,365","1,376","1,376"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250302,2025-04-10 08:00:22,0,"1,603","1,211","1,211","1,193","1,211","1,211"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250303,2025-04-10 08:00:22,0,"1,737","1,392","1,392","1,373","1,392","1,392"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250304,2025-04-10 08:00:22,0,"1,789","1,449","1,449","1,441","1,449","1,449"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250305,2025-04-10 08:00:22,0,"1,733","1,417","1,417","1,402","1,417","1,417"
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250306,2025-04-10 08:00:22,0,"1,726","1,408","1,408","1,400","1,408","1,408"
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250307,2025-04-10 08:00:22,0,"1,657","1,348","1,348","1,338","1,348","1,348"
7,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250308,2025-04-10 08:00:22,0,"1,806","1,334","1,334","1,323","1,334","1,334"
8,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250309,2025-04-10 08:00:22,0,"1,711","1,265","1,265","1,259","1,265","1,265"
9,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250310,2025-04-10 08:00:22,0,"1,826","1,418","1,418","1,410","1,418","1,418"


In [20]:
''' TB4S001400CS : TVS Now Gross Adds '''

v_tm_key_day = 20250301
v_metric_cd = 'TB4S001400CS'
print(f'\nTB4S001400CS : TVS Now Gross Adds\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4S001400CS_day_df = chk_src_df.copy()
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['METRIC_CD']==v_metric_cd]
TB4S001400CS_day_df = TB4S001400CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4S001400CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4S001400CS_day_df[col] = TB4S001400CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4S001400CS_day_df


TB4S001400CS : TVS Now Gross Adds

   -> v_metric_cd: TB4S001400CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TVS,TB4S001400CS,TVS Now Gross Adds,20250301,2025-04-10 08:11:55,0,24,24,24,24,24,24
1,TVS,TB4S001400CS,TVS Now Gross Adds,20250302,2025-04-10 08:11:55,0,47,47,47,47,47,47
2,TVS,TB4S001400CS,TVS Now Gross Adds,20250303,2025-04-10 08:11:55,0,22,22,22,22,22,22
3,TVS,TB4S001400CS,TVS Now Gross Adds,20250304,2025-04-10 08:11:55,0,23,23,23,23,23,23
4,TVS,TB4S001400CS,TVS Now Gross Adds,20250305,2025-04-10 08:11:55,0,79,79,79,79,79,79
5,TVS,TB4S001400CS,TVS Now Gross Adds,20250306,2025-04-10 08:11:55,0,215,215,215,215,215,215
6,TVS,TB4S001400CS,TVS Now Gross Adds,20250307,2025-04-10 08:11:55,0,162,162,162,162,162,162
7,TVS,TB4S001400CS,TVS Now Gross Adds,20250308,2025-04-10 08:11:55,0,143,143,143,143,143,143
8,TVS,TB4S001400CS,TVS Now Gross Adds,20250309,2025-04-10 08:11:55,0,145,145,145,145,145,145
9,TVS,TB4S001400CS,TVS Now Gross Adds,20250310,2025-04-10 08:11:55,0,87,87,87,87,87,87


## Actual : Issue
->  DIM_CORP_KPI

In [21]:
# ''' Execute Issue by case '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# # v_start_date = 20250101
# print(f'\nParameter input...\n')
# # print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ T.TM_KEY_DAY AS TM_KEY_MTH
#         , T.METRIC_CD, T.METRIC_NAME, T.AREA_TYPE, T.AREA_CD, T.AREA_DESC
#         , T.METRIC_VALUE AS TARGET
#         , A.ACTUAL
#         , O.ORGID_P, O.ZONE_TYPE, O.ORGID_G, O.TDS_SGMD, O.ORGID_H, O.HOP_HINT, O.ORGID_HH, O.D_CLUSTER
        
#     FROM CDSAPPO.DIM_CORP_TARGET T

#     LEFT JOIN (
#         SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#             , PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#             , SUM(METRIC_VALUE) AS ACTUAL, MAX(LOAD_DATE) LOAD_DATE
#         FROM CDSAPPO.DIM_CORP_KPI
#         WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
#         AND AREA_TYPE = 'CCAA'
#         AND TM_KEY_DAY >= 20250101
#         GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#     ) A
#         ON A.METRIC_CD = T.METRIC_CD
#         AND A.AREA_CD = T.AREA_CD
#         AND A.TM_KEY_MTH = T.TM_KEY_DAY
        
#     LEFT JOIN (
#         SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
#             , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG 
#             , SUBSTR(CCAATT,1,4) AS CCAA, DISTRICT_EN
#         FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
#         WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
#     ) O
#         ON O.CCAA = T.AREA_CD

#     WHERE T.METRIC_CD = 'TB3S000102CS'
#     AND T.AREA_TYPE = 'CCAA'
#     AND T.TM_KEY_DAY >= 202501

#     --ORDER BY T.TM_KEY_DAY, T.METRIC_CD, T.AREA_TYPE, O.ZONE_TYPE, O.ORGID_G, O.ORGID_H, O.ORGID_HH
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Summary DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [22]:
# ''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

# v_tm_key_mth = 202501
# # v_metric_cd = 'TB3S000102CS'
# print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
# # print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

# TB3S000102CS_issue_df = chk_src_df.copy()
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'] > 0]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'].isna()]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['METRIC_CD']==v_metric_cd]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)

# mod_col_list = TB3S000102CS_issue_df.iloc[:, 6:8].columns.tolist()
# for col in mod_col_list:
#     TB3S000102CS_issue_df[col] = TB3S000102CS_issue_df[col].apply(lambda x: format(x, ',.0f'))
    
# # TB3S000102CS_issue_df
# print(f'\n{TB3S000102CS_issue_df.to_string(max_cols=100)}')

## Output File
->  DIM_CORP_KPI

In [23]:
# ''' Execute Sales Monthly '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20240101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ 
#         SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#         , CASE WHEN REGEXP_LIKE(METRIC_CD, 'CS$|CG$') THEN 'Y2025' ELSE 'Y2024' END VER_FLAG
#         , PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P_ACTUAL
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
#         , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) "016"
#         , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) "040"
#         , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) "080"
#         , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
#     FROM CDSAPPO.DIM_CORP_KPI
#     WHERE METRIC_CD NOT LIKE 'Error%'
#     AND EXISTS (
#         SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
#         FROM (
#             SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#         ) TMP
#         WHERE TMP_CD = AREA_CD )
#     AND TM_KEY_DAY >= {v_start_date}
#     GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#     --ORDER BY 2, 3, PRODUCT, COMP_CD, METRIC_CD 
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Sales Monthly DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [24]:
# ''' Generate CSV file '''

# op_dir = 'data'
# op_monthly_file = f'sales_monthly_{str_curr_dt}'

# monthly_sales_df.to_csv(f'{op_dir}/tmp_{op_monthly_file}.csv', index=False, encoding='utf-8')
# print(f'\n   -> Generate "tmp_{op_monthly_file}.csv" successfully')

# monthly_sales_df.tail(3)